In [1]:
import argparse
import numpy as np
import os
import cv2
import tqdm
import torch
import pickle
import copy

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.utils.logger import setup_logger

import perspective2d.modeling  # noqa
from perspective2d.utils.predictor import VisualizationDemo
from perspective2d.config import get_perspective2d_cfg_defaults
from perspective2d.utils import draw_perspective_fields, draw_from_r_p_f_cx_cy
from perspective2d.utils.panocam import PanoCam
import matplotlib.pyplot as plt

/home/jinlinyi/.pyenv/versions/anaconda3-2022.10spdrdf/envs/perspective/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jinlinyi/.pyenv/versions/anaconda3-2022.10spdrdf/envs/perspective/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
MODEL_ID = 'paramnet_gsv_rpfpp' # 'paramnet_gsv_rpfpp'

In [3]:
model_zoo = {
    # trained on GSV dataset, predicts Perspective Fields + camera parameters (roll, pitch, fov, principal point)
    'paramnet_gsv_rpfpp': {
        'weights': ['https://www.dropbox.com/s/ufdadxigewakzlz/paramnet_gsv_rpfpp.pth'],
        'opts': ['MODEL.WEIGHTS', 'models/paramnet_gsv_rpfpp.pth'],
        'config_file': 'models/paramnet_gsv_rpfpp.yaml',
        'param': True,
    },
    # trained on GSV dataset, predicts Perspective Fields + camera parameters (roll, pitch, fov), assuming centered principal point
    'paramnet_gsv_rpf': {
        'weights': ['https://www.dropbox.com/s/g6xwbgnkggapyeu/paramnet_gsv_rpf.pth'],
        'opts': ['MODEL.WEIGHTS', 'models/paramnet_gsv_rpf.pth'],
        'config_file': 'models/paramnet_gsv_rpf.yaml',
        'param': True,
    },
}


In [ ]:


assert MODEL_ID in model_zoo.keys()

for html in model_zoo[MODEL_ID]['weights']:
    if not os.path.exists(os.path.join('models', html.split('/')[-1])):
        !wget -P models/ {html}
def setup_cfg(args):
    cfgs = []
    configs = args['config_file'].split('#')
    weights_id = args['opts'].index('MODEL.WEIGHTS') + 1
    weights = args['opts'][weights_id].split('#')
    for i, conf in enumerate(configs):
        if len(conf) != 0:
            tmp_opts = copy.deepcopy(args['opts'])
            tmp_opts[weights_id] = weights[i]
            cfg = get_cfg()
            get_perspective2d_cfg_defaults(cfg)
            cfg.merge_from_file(conf)
            cfg.merge_from_list(tmp_opts)
            cfg.freeze()
            cfgs.append(cfg)
    return cfgs

perspective_cfg_list = setup_cfg(model_zoo[MODEL_ID])
demo_perspective = VisualizationDemo(cfg_list=perspective_cfg_list)

## Input image

In [ ]:
path = '../assets/imgs/cityscape.jpg'
img = read_image(path, format="BGR")
plt.imshow(img[:,:,::-1])
plt.title("input")
plt.show()

## Inference PerspectiveNet + ParamNet (See Paper Fig 3)

In [ ]:
pred = demo_perspective.run_on_image(img)
if 'pred_general_vfov' not in pred.keys():
    pred['pred_general_vfov'] = pred['pred_vfov']
if 'pred_rel_cx' not in pred.keys():
    pred['pred_rel_cx'] = torch.FloatTensor([0])
if 'pred_rel_cy' not in pred.keys():
    pred['pred_rel_cy'] = torch.FloatTensor([0])
    
r_p_f_rad = np.radians(
    [
        pred['pred_roll'].cpu().item(),
        pred['pred_pitch'].cpu().item(),
        pred['pred_general_vfov'].cpu().item(),
    ]
)
cx_cy = [
    pred['pred_rel_cx'].cpu().item(),
    pred['pred_rel_cy'].cpu().item(),
]


## Visualize predictions

In [ ]:
print(pred)
predicted_param_vis = draw_from_r_p_f_cx_cy(
    img[:,:,::-1], 
    *r_p_f_rad,
    *cx_cy,
    'rad',
    up_color=(0,1,0),
)
plt.imshow(predicted_param_vis)
plt.title("From Predicted Parameters")
plt.show()